In [38]:
import sys
sys.path.append("/home/sara/lib")
# sys.path

In [39]:
#!/usr/bin/python
import csv
import gc
import numpy as np
import pylab as pl
import pandas as pd
from time import time
import libopf_py
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn import datasets

In [40]:
def read_csv():
    file = pd.read_csv("csv_dataset/dataframe.csv", header=None, sep=';')
    return file

In [41]:
def read_dataset():
    df = read_csv()
    X = df.iloc[1:665,1:137]
    y = df.iloc[1:,137]
    #X, y = datasets.load_breast_cancer(return_X_y=True)
    return X, y
    #benchmark(X, y, len(y))

Vendo qual o formato dos dados que entram no programa

In [42]:
datasets.load_breast_cancer(return_X_y=True)

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

Como o programa separa os dados?

In [43]:
X, y = datasets.load_breast_cancer(return_X_y=True)
print(X)
print(y)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

Como eu leio meus dados?

In [44]:
df = read_csv()
print(df)
len(df)

           0         1         2         3         4         5         6    \
0          NaN  land_1_x  land_1_y  land_2_x  land_2_y  land_3_x  land_3_y   
1      frame_0      84.0     213.0      88.0     263.0      96.0     312.0   
2     frame_10      82.0     217.0      87.0     268.0      96.0     317.0   
3     frame_20      89.0     221.0      94.0     271.0     102.0     319.0   
4     frame_30      83.0     224.0      89.0     274.0      97.0     324.0   
5     frame_40      80.0     217.0      84.0     267.0      91.0     316.0   
6     frame_50      82.0     220.0      87.0     269.0      94.0     317.0   
7     frame_60      93.0     224.0      97.0     272.0     104.0     320.0   
8     frame_70      92.0     228.0      97.0     276.0     105.0     322.0   
9     frame_80      78.0     210.0      82.0     262.0      90.0     312.0   
10    frame_90      85.0     217.0      89.0     266.0      97.0     314.0   
11   frame_100      96.0     232.0     100.0     281.0     108.0

666

Tiro a primeira coluna e a primeira linha

In [45]:
df = df.drop([0], axis=0)
df = df.drop(columns=[0])
df.head()

,1,2,3,4,5,6,7,8,9,10,...,128,129,130,131,132,133,134,135,136,137
1,84.0,213.0,88.0,263.0,96.0,312.0,103.0,360.0,115.0,408.0,...,406.0,318.0,405.0,282.0,405.0,265.0,407.0,246.0,404.0,1
2,82.0,217.0,87.0,268.0,96.0,317.0,104.0,366.0,117.0,414.0,...,394.0,321.0,405.0,284.0,422.0,265.0,424.0,246.0,422.0,1
3,89.0,221.0,94.0,271.0,102.0,319.0,110.0,368.0,122.0,416.0,...,396.0,326.0,403.0,289.0,414.0,270.0,416.0,250.0,414.0,1
4,83.0,224.0,89.0,274.0,97.0,324.0,104.0,373.0,117.0,421.0,...,394.0,333.0,402.0,291.0,433.0,269.0,436.0,245.0,433.0,1
5,80.0,217.0,84.0,267.0,91.0,316.0,96.0,365.0,108.0,413.0,...,385.0,336.0,385.0,287.0,417.0,262.0,419.0,236.0,416.0,1


Tratamento dos meus dados para ficarem como devem

In [46]:
X, y = read_dataset() 
X = X.to_numpy()
X = X.astype('float64')
print(X)
y = y.to_numpy()
y = y.astype('float64')
print(y)
print("tamanho:", len(y))

[[ 84. 213.  88. ... 407. 246. 404.]
 [ 82. 217.  87. ... 424. 246. 422.]
 [ 89. 221.  94. ... 416. 250. 414.]
 ...
 [ 91. 234.  97. ... 421. 254. 419.]
 [ 82. 225.  88. ... 405. 236. 403.]
 [ 94. 220.  98. ... 398. 250. 396.]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.

In [49]:
def benchmark(data, target, n_samples):
    list_n_samples = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    opf_results = np.zeros((len(list_n_samples), 4))
    svm_results = np.zeros((len(list_n_samples), 4))
    bayes_results = np.zeros((len(list_n_samples), 4))
    linear_results = np.zeros((len(list_n_samples), 4))
    sgd_results = np.zeros((len(list_n_samples), 4))
    tree_results = np.zeros((len(list_n_samples), 4))

    for i, size in enumerate(list_n_samples):
        n_split = int(size * n_samples)
        rand = np.random.permutation(n_samples)
        random_data = data[rand]
        random_label = target[rand]
        data_train, data_test = random_data[:n_split], random_data[n_split:]
        label_train, label_test = random_label[:n_split], random_label[n_split:]

        def _opf():
            label_train_32 = label_train.astype(np.int32)
            label_test_32 = label_test.astype(np.int32)
            O = libopf_py.OPF()
            t = time()
            O.fit(data_train, label_train_32)

            opf_results[i, 3] = time() - t
            t = time()
            print("----------OPF------------")
            print(t)
            predicted = O.predict(data_test)
            opf_results[i, 0] = precision_score(label_test_32, predicted, average='binary')
            opf_results[i, 1] = recall_score(label_test_32, predicted, average='binary')
            opf_results[i, 2] = f1_score(label_test_32, predicted, average='binary')
            gc.collect()

        def _svm():
            clf = svm.SVC(C=1000, gamma='auto')
            t = time()
            print("-----------SVM-----------")
            print(t)
            clf.fit(data_train, label_train)
            svm_results[i, 3] = time() - t
            predicted = clf.predict(data_test)
            svm_results[i, 0] = precision_score(label_test, predicted, average='binary')
            svm_results[i, 1] = recall_score(label_test, predicted, average='binary')
            svm_results[i, 2] = f1_score(label_test, predicted, average='binary')
            gc.collect()

        def _bayes():
            clf = GaussianNB()
            t = time()
            print("-----------BAYES-----------")
            print(t)
            clf.fit(data_train, label_train)
            bayes_results[i, 3] = time() - t
            predicted = clf.predict(data_test)
            bayes_results[i, 0] = precision_score(label_test, predicted, average='binary')
            bayes_results[i, 1] = recall_score(label_test, predicted, average='binary')
            bayes_results[i, 2] = f1_score(label_test, predicted, average='binary')
            gc.collect()

        def _linear():
            clf = LogisticRegression(C=1, penalty='l2', solver='liblinear')
            t = time()
            print("-----------LINEAR-----------")
            print(t)
            clf.fit(data_train, label_train)
            linear_results[i, 3] = time() - t
            predicted = clf.predict(data_test)
            linear_results[i, 0] = precision_score(label_test, predicted, average='binary')
            linear_results[i, 1] = recall_score(label_test, predicted, average='binary')
            linear_results[i, 2] = f1_score(label_test, predicted, average='binary')
            gc.collect()

        def _sgd():
            clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3)
            t = time()
            print("-----------SGD-----------")
            print(t)
            clf.fit(data_train, label_train)
            linear_results[i, 3] = time() - t
            predicted = clf.predict(data_test)
            sgd_results[i, 0] = precision_score(label_test, predicted, average='binary')
            sgd_results[i, 1] = recall_score(label_test, predicted, average='binary')
            sgd_results[i, 2] = f1_score(label_test, predicted, average='binary')
            gc.collect()

        def _tree():
            clf = tree.DecisionTreeClassifier()
            t = time()
            print("-----------TREE-----------")
            print(t)
            clf.fit(data_train, label_train)
            tree_results[i, 3] = time() - t
            predicted = clf.predict(data_test)
            tree_results[i, 0] = precision_score(label_test, predicted, average='binary')
            tree_results[i, 1] = recall_score(label_test, predicted, average='binary')
            tree_results[i, 2] = f1_score(label_test, predicted, average='binary')
            gc.collect()

        _opf()
        _svm()
        _bayes()
        _linear()
        _sgd()
        _tree()

    pl.figure()
    pl.plot(list_n_samples, opf_results[:, 2], label="OPF")
    pl.plot(list_n_samples, svm_results[:, 2], label="SVM RBF")
    pl.plot(list_n_samples, bayes_results[:, 2], label="Naive Bayes")
    pl.plot(list_n_samples, linear_results[:, 2], label="Logistic Regression")
    pl.plot(list_n_samples, sgd_results[:, 2], label="SGD")
    pl.plot(list_n_samples, tree_results[:, 2], label="Decision Trees")
    pl.legend(loc='lower right', prop=dict(size=8))
    pl.xlabel("Training set size")
    pl.ylabel("F1 score")
    pl.title("Precision")
    pl.show()

Note que abaixo o programa chama essa função que está com a chamada para benchmark() desativada. Por isso o programa não reclama por enquanto das várias classes.

In [50]:
read_dataset()

(       1      2      3      4      5      6      7      8      9      10   \
 1     84.0  213.0   88.0  263.0   96.0  312.0  103.0  360.0  115.0  408.0   
 2     82.0  217.0   87.0  268.0   96.0  317.0  104.0  366.0  117.0  414.0   
 3     89.0  221.0   94.0  271.0  102.0  319.0  110.0  368.0  122.0  416.0   
 4     83.0  224.0   89.0  274.0   97.0  324.0  104.0  373.0  117.0  421.0   
 5     80.0  217.0   84.0  267.0   91.0  316.0   96.0  365.0  108.0  413.0   
 6     82.0  220.0   87.0  269.0   94.0  317.0   99.0  366.0  112.0  413.0   
 7     93.0  224.0   97.0  272.0  104.0  320.0  110.0  369.0  123.0  417.0   
 8     92.0  228.0   97.0  276.0  105.0  322.0  113.0  371.0  126.0  417.0   
 9     78.0  210.0   82.0  262.0   90.0  312.0   97.0  363.0  111.0  410.0   
 10    85.0  217.0   89.0  266.0   97.0  314.0  103.0  363.0  115.0  411.0   
 11    96.0  232.0  100.0  281.0  108.0  329.0  115.0  377.0  128.0  425.0   
 12    78.0  223.0   82.0  272.0   89.0  320.0   96.0  369.0  11